In [1]:
import os

from argparse import ArgumentParser
import pandas as pd

from mapc_sim.constants import TAU

N_RUNS = 10
N_STEPS = 600
SIM_TIME = N_STEPS * TAU

In [ ]:
agent_dir = "/Users/wciezobka/ncn/mapc-optimal-research/results/out_server/dcf/residential"

# Read results
results_files = os.listdir(agent_dir)
results_files = [f for f in results_files if f.endswith(".csv")]
results_files = sorted(results_files)
results_dfs = {int(f.split('_')[2]) - 100: pd.read_csv(os.path.join(agent_dir, f)) for f in results_files}

# Calculate total throughput
get_total_thr = lambda df: (df["AMPDUSize"].sum() * 1e-6 / SIM_TIME / N_RUNS)
total_thr = {f: get_total_thr(df) for f, df in results_dfs.items()}

# Calculate worst case throughput
get_worst_case_thr = lambda df: (df.groupby("Dst")["AMPDUSize"].sum() * 1e-6 / SIM_TIME / N_RUNS).min()
worst_case_thr = {f: get_worst_case_thr(df) for f, df in results_dfs.items()}

In [9]:
results_dfs[0]

,SimTime,RunNumber,FrameID,Retransmission,Src,Dst,AMPDUSize,MCS,CW,Backoff,Collision,NSuccess,NCollision
0,0.005499,10,0,0,0,2,0,11,16,0,True,0,65
1,0.005499,10,0,0,15,18,0,11,16,0,True,0,65
2,0.011007,10,0,0,5,8,780000,11,16,3,False,65,0
3,0.016524,10,1,0,5,9,0,11,16,4,True,0,65
4,0.016524,10,0,1,15,18,0,11,32,9,True,0,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6785,3.271533,3,128,2,10,14,780000,11,64,58,False,65,0
6786,3.277049,3,142,0,15,19,780000,11,16,13,False,65,0
6787,3.282557,3,134,0,0,2,0,11,16,12,True,0,65
6788,3.282558,3,130,0,5,9,0,11,16,11,True,0,65


In [31]:
get_link_rates = lambda df: df.groupby("Dst")["AMPDUSize"].sum() * 1e-6 / SIM_TIME / N_RUNS
get_txops = lambda df: df.groupby("Dst")["AMPDUSize"].count()
get_psucc = lambda df: df.groupby("Dst")["NSuccess"].sum() / (df.groupby("Dst")["NSuccess"].sum() + df.groupby("Dst")["NCollision"].sum())

In [41]:
scenario_id = 3

# Get link rates and txops and combine them to a dataframe
link_rates = get_link_rates(results_dfs[scenario_id])
txops = get_txops(results_dfs[scenario_id])
psucc = get_psucc(results_dfs[scenario_id])
combined = pd.concat([link_rates, txops, psucc], axis=1)
combined.columns = ["LinkRate", "TxOps", "PSucc"]

print(f"Worst Case Throughput: {worst_case_thr[scenario_id]}")
display(combined)


Worst Case Throughput: 0.0711159737417943


,LinkRate,TxOps,PSucc
Dst,,,
1,26.734136,1131,0.997143
2,24.646973,1047,0.993050
3,25.909920,1097,0.996354
4,26.728665,1129,0.998705
6,0.238512,16,0.628846
7,0.071116,5,0.600000
8,0.213348,13,0.692308
9,0.260759,19,0.578947
11,5.852298,326,0.757291


In [38]:
worst_case_thr

{0: 6.708606856309262,
 1: 0.2476294675419402,
 2: 0.032458059810357404,
 3: 0.0711159737417943,
 4: 0.686725018234865,
 5: 0.17104303428154632,
 6: 0.274981765134938,
 7: 0.17906637490882565,
 8: 0.4201312910284464,
 9: 0.4168490153172867,
 10: 0.4401896425966448}